# 手写数字识别

1. 引入需要的库

In [1]:
import numpy as np
import scipy.special    #使用sigmoid函数，expit()
from sklearn.model_selection import KFold
import random

2. 创建神经网络

In [3]:
class neuralNetwork():
#初始化神经网络，输入层、隐藏层、输出层、学习率
    def __init__(self,inputnodes,hiddennodes,outputnodes,learninggrate):
        self.inodes=inputnodes
        self.onodes=outputnodes
        self.hnodes=hiddennodes
        #学习率
        self.lr=learninggrate
        #链接权重矩阵
        self.wih=(np.random.rand(self.hnodes,self.inodes)-0.5)
        self.who=(np.random.rand(self.onodes,self.hnodes)-0.5)
        #sigmoid激活函数
        self.activation_function=lambda x:scipy.special.expit(x)
    #训练神经网络
    def train(self,inputs_list,targets_list):
        inputs=np.array(inputs_list,ndmin=2).T      #T求转置，得到列向量
        targets=np.array(targets_list,ndmin=2).T
        
        hidden_inputs=np.dot(self.wih,inputs)
        hidden_outputs=self.activation_function(hidden_inputs)
        final_inputs=np.dot(self.who,hidden_outputs)
        final_outputs=self.activation_function(final_inputs)
        
        output_errors=targets-final_outputs
        hidden_errors=np.dot(self.who.T,output_errors)
        self.who+=self.lr*np.dot((output_errors*final_outputs*(1.0-final_outputs)),np.transpose(hidden_outputs))#改变索引，不改变矩阵形状
        self.wih+=self.lr*np.dot((hidden_errors*hidden_outputs*(1.0-hidden_outputs)),np.transpose(inputs))
    #测试结果
    def query(self,inputs_list):
        inputs=np.array(inputs_list,ndmin=2).T
        hidden_inputs=np.dot(self.wih,inputs)
        hidden_outputs=self.activation_function(hidden_inputs)
        final_inputs=np.dot(self.who,hidden_outputs)
        final_outputs=self.activation_function(final_inputs)
        return final_outputs

3. 调节参数

In [4]:
inodes=784
onodes=10
hnodes=200
learningrate=0.1
kf = KFold(n_splits=5,random_state=None)    #k折交叉验证

4. 读取并简单处理train数据集，创建数据迭代器

In [5]:
with open("train.csv",'r') as f:
    data=f.readlines()
data_list=data[1:]
epochs=3
index=kf.split(data_list)

5. 开始验证模型性能（采用五折交叉验证，进行了三次迭代）

训练需要一定时间

In [5]:
total_score=[]
for train_index, test_index in index:
    #五种不同的训练集
    n=neuralNetwork(inodes,hnodes,onodes,learningrate)
    #每种训练集进行三次迭代
    for e in range(epochs):
        for train in train_index:
            value=data_list[train].split(',')
            inputs=(np.asfarray(value[1:])/255.0*0.99)+0.01
            targets=np.zeros(onodes)+0.01
            targets[int(value[0])]=0.99
            n.train(inputs,targets)
        #训练完成后测试
        if(e==epochs-1):
            score=0
            for test in test_index:
                rvalue=data_list[test].split(',')
                rinputs=(np.asfarray(rvalue[1:])/255.0*0.99)+0.01
                outputs=n.query(rinputs)
                label=np.argmax(outputs)
                correct_label=int(rvalue[0])
                if(label==correct_label):
                    score+=1.0
            #获得迭代三次后的神经网络的准确率
            total_score.append(score/len(test_index))

6. 输出模型性能

In [6]:
#对五种数据集得到的准确率求平均
print(np.mean(total_score))

0.966047619047619


7. 重新训练模型

In [6]:
n=neuralNetwork(inodes,hnodes,onodes,learningrate)
train=data_list[:33601]
test=data_list[33601:]
for e in range(epochs):
    for tr in train:
        value=tr.split(',')
        inputs=(np.asfarray(value[1:])/255.0*0.99)+0.01
        targets=np.zeros(onodes)+0.01
        targets[int(value[0])]=0.99
        n.train(inputs,targets)

8. 随机挑个图片看看预测结果

In [9]:
ti=random.randint(0, len(data_list)-1)
te=data_list[ti]
rvalue=te.split(',')
rinputs=(np.asfarray(rvalue[1:])/255.0*0.99)+0.01
outputs=n.query(rinputs)
label=np.argmax(outputs)
correct_label=int(rvalue[0])
print(f"预测结果为{label} 实际答案为{correct_label}")

预测结果为5 实际答案为5
